In [1]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [11]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
import time

# Load your dataset
filename = "NF-BOT-IOT"
train_data = pd.read_csv('/content/drive/MyDrive/dataset/NF-BOT-IOT_train_preprocessed.csv', sep=',', encoding='utf-8')
test_data = pd.read_csv('/content/drive/MyDrive/dataset/NF-BOT-IOT_test_preprocessed.csv', sep=',', encoding='utf-8')

# Prepare training data
X_train = train_data.drop(columns=['label'], axis=1)
y_train = train_data['label']

# Prepare test data
X_test = test_data.drop(columns=['label'], axis=1)
y_test = test_data['label']

# Split a small portion of training data for optimization (Poor and Rich Optimization)
X_t, _, y_t, _ = train_test_split(X_train, y_train, train_size=0.01, random_state=7)
X_test_t, _, y_test_t, _ = train_test_split(X_test, y_test, train_size=0.01, random_state=7)

# Define custom Poor and Rich Optimization (PARO) algorithm for feature selection
class PARO:
    def __init__(self, num_agents=5, max_iter=10):
        self.num_agents = num_agents
        self.max_iter = max_iter

    def fit(self, X, y):
        start_time = time.time()  # Start the timer
        num_features = X.shape[1]
        poor_agents = np.ones(num_features)  # Initialize motivation levels for poor agents
        rich_agent = np.random.choice([True, False], size=num_features)  # Randomly initialize the rich agent's selection
        best_feature_set = rich_agent
        best_accuracy = self._evaluate(X, y, best_feature_set)

        for iteration in range(self.max_iter):
            # Poor agents
            for agent in range(self.num_agents):
                # Generate a new candidate solution for poor agents by flipping a random feature
                new_agent = poor_agents.copy()
                flip_idx = np.random.randint(num_features)
                new_agent[flip_idx] = not new_agent[flip_idx]

                # Evaluate the new candidate solution
                new_accuracy = self._evaluate(X, y, new_agent)
                delta_accuracy = new_accuracy - best_accuracy

                # Update poor agent's selection based on improvement
                if delta_accuracy > 0:
                    poor_agents = new_agent.copy()
                    best_accuracy = new_accuracy
                    best_feature_set = new_agent.copy()

            # Rich agent
            # Generate a new candidate solution for rich agent by flipping a random feature
            flip_idx = np.random.randint(num_features)
            rich_agent[flip_idx] = not rich_agent[flip_idx]

            # Evaluate the new candidate solution
            new_accuracy = self._evaluate(X, y, rich_agent)
            delta_accuracy = new_accuracy - best_accuracy

            # Update rich agent's selection based on improvement
            if delta_accuracy > 0:
                best_accuracy = new_accuracy
                best_feature_set = rich_agent.copy()

        end_time = time.time()  # End the timer
        execution_time = end_time - start_time   # Calculate execution time

        self.selected_features = best_feature_set
        self.execution_time = execution_time
        self.num_selected_features = np.sum(best_feature_set)  # Number of selected features
        return self

    def _evaluate(self, X, y, selected_features):
        clf = RandomForestClassifier()
        X_subset = X.iloc[:, selected_features]
        clf.fit(X_subset, y)
        accuracy = accuracy_score(y, clf.predict(X_subset))
        return accuracy

# Instantiate and run the Poor and Rich Optimization algorithm for feature selection on training data
paro = PARO(num_agents=5, max_iter=10)
paro.fit(X_t, y_t)

# Apply selected features to training data
selected_features_train = X_train.columns[paro.selected_features]

# Print number of selected features and list them for the training data
num_selected_features_train = paro.num_selected_features
print("Number of selected features (training data):", num_selected_features_train)
print("Selected Features (training data):", selected_features_train.tolist())

# Use selected features to filter columns in X_train
X_train_selected = X_train[selected_features_train]

# Output the dataframe with selected features for the training data
print("DataFrame with selected features for training data:")
print(X_train_selected.head())

# Apply selected features to test data
selected_features_test = X_test.columns[paro.selected_features]

# Print number of selected features and list them for the test data
num_selected_features_test = num_selected_features_train  # Same as training data for test data
print("Number of selected features (test data):", num_selected_features_test)
print("Selected Features (test data):", selected_features_test.tolist())

# Use selected features to filter columns in X_test
X_test_selected = X_test[selected_features_test]

# Output the dataframe with selected features for the test data
print("DataFrame with selected features for test data:")
print(X_test_selected.head())

# Create a DataFrame for optimization results
optimization_results = pd.DataFrame({
    "Optimization": ["PoorAndRichOptimization"],
    "Execution Time of Optimizer": [paro.execution_time],
    "No of Feature Selected": [num_selected_features_train],
    "Selected Feature": [', '.join(selected_features_train.tolist())]
})

# Save optimization results to a CSV file
optimization_results.to_csv('NF-BOT-IOT_PoorAndRichOptimization_feature.csv', index=False)


Number of selected features (training data): 7
Selected Features (training data): ['L4_SRC_PORT', 'L4_DST_PORT', 'PROTOCOL', 'L7_PROTO', 'IN_BYTES', 'OUT_BYTES', 'FLOW_DURATION_MILLISECONDS']
DataFrame with selected features for training data:
   L4_SRC_PORT  L4_DST_PORT  PROTOCOL  L7_PROTO  IN_BYTES  OUT_BYTES  \
0     0.165107    -0.331298 -0.233340 -0.252892 -0.018640  -0.007950   
1    -3.856688    -0.565481 -0.233340  7.100761 -0.019469  -0.008504   
2     0.194481    -0.331298 -0.233340 -0.252892 -0.018640  -0.007950   
3     0.306015     2.671561  4.057829 -0.252892 -0.019494  -0.008553   
4     0.151803    -0.331298 -0.233340 -0.252892 -0.018640  -0.007950   

   FLOW_DURATION_MILLISECONDS  
0                    0.495722  
1                    0.495841  
2                    0.495720  
3                   -2.083921  
4                    0.495721  
Number of selected features (test data): 7
Selected Features (test data): ['L4_SRC_PORT', 'L4_DST_PORT', 'PROTOCOL', 'L7_PROTO', 'I